# Object oriented programming in Python

download the lecture 8 folder from Cybox

- I want to quickly get to Class design, so the my intro has to be quick and dirty!
- I will only talk about Encapsulation and Inheritance
- gentler intro to OOP with many more details at HCI 574 lectures 27 thru 29


### Encapsulation
- classes create a linkage of data (any object), called attributes or properties with methods (functions)
- an object (instance) is instantiated from a class initialized via the `__init__()` method (*)
- the object  has is its own namespace, it must use __self__ to access its own attributes and methods (classes also define their own namespace)
- there are many _magic methods_ that use overloading to make your class work like a standard python object (see __str__ for print() support)
- https://towardsdatascience.com/how-to-be-fancy-with-python-part-2-70fab0a3e492



- For the examples I have used draw.io show class relationships (somewhat like UML but very sloppy!)
- I used https://app.diagrams.net and the Draw.io Integration Extension for VS Code (experimental?)


In [29]:
# class def and object instantiation example

class Person(object): # all base classes should be derived (inherit) from the standard python class object 

    # create a instance (object) of the class Person
    def __init__(self, nm, ag):  # like a function but MUST always have self as 1. arg!
        self.name = nm  # attribute, created in self namespace (self.<attrib>)
        self.age = ag
        self._pets_owned = [] # _ means "protected" 
        # no return !

    # overloading str() with __str__ => what happens when object is print()ed?
    def __str__(self):  # must have self!
        return "Name: " + self.name + ", age: " + str(self.age) # should add pets

    # age the age (aged up)
    def set_age(self, new_ag):
        self.age = new_ag

# this class definition would typically go into a module e.g. person_class.py

In [30]:
# from person_class import Person  # how to import a class from a module

# instantiate an object
bob = Person("Bob", 35)  # matches nm, ag in __init__
print(bob) # calls bob's __str__() method

# age up
bob.set_age(36)
print("Info for:", bob)

Name: Bob, age: 35
Info for: Name: Bob, age: 36


- Python does not have any real access restrictions (friend, private, protected, etc.)
- you can indicate that attributes are NOT to be altered from the outside (_pets_owned) but this is not enforced
- its also legal to get/set attributes directly, but it's advised to write get/set __access methods__ to hide the details and decouple that attribute names from the user - if I were to change self.name to self.name_of_person the code below would no longer work! 

In [31]:
# direct attribute access
bob.name = "Bobby"
print(bob.name, bob.age)
bob._pets_owned = ["cat", "dog"]

Bobby 36


### Inheritance
- a (super) class inherits all methods from a base class
- but: you must implicitly call the `__init__` of the base class to create (inherit) its attributes!

In [32]:
class Worker(Person): # inherits from Person

    def __init__(self, nme, ag, slry): # also has a salary
        self.salary = slry

        # we do NOT make name and age attributes here b/c Person's __init__ can do that
        # that for us! Person.<method> => this is a class method
        Person.__init__(self, nme, ag)

        print("DEBUG", self.name) # proof that we now have a name attrib from Person

    # It does NOT define __str__ or set_age() !

In [33]:
jane = Worker("Jane", 45, 6753)
jane.set_age(46) # uses Person.set_age()
print(jane) # uses Person.__str__()

DEBUG Jane
Name: Jane, age: 46


# How to design classes


This example is heavily based on:  https://towardsdatascience.com/a-data-scientist-should-know-at-least-this-much-python-oop-d63f37eaac4d


- I ended up massively re-writing the example but did not fully clean up the dubious naming scheme
- sorry for the mix of guest_list (my preference!) and guestList (not the offical python style)
- Class names should start with upper case
- The scenario is a bit contrived but has many things we can improve upong



### PYTHON OOP: Design of the Reservation Management System 
- Villa Vacation Scenario : 
    - guest (1+ people) rents a villa for a perid of time 
    - each Villa gets a personal assistant (PA)
    - there's a standard and a VIP Villa  
    - VIP villas have a better, snootier VIP personal assistant!

    
- Object model:
    - create a central resort object
    - define what villas it contains and which people work at it as personal assistants
    - create guest objects (capture some guest data, say on check in)

    - Reserve a villa for a period of time (this could be done pre guest check in)
    - Book (connect) a guest into a reserved villa
    - get info on:
        - what villas does the resort own?
        - which villas house which guests (which are VIP?)
        - where are our guests staying (which villa?)  


- 2 types of linkages: reservation with villa and guest with reservation
- Assumed work flow:
    - villa gets reserved for a time (results in a reservation object and a human readable reservation id, which we could think of as a confirmation code)
    - guest checks in
    - guest gets booked into and earlier reservation

### villa

- Class villa encapsulates the following rented villa characteristics
    - name of villa -
    - name of personal assistant. 
    - offers functions that inform about the 
        - hours that the personal assistant will be on call - 
        - the dates that the villa will be cleaned and keys will be changed. (end of stay)
    - get the label of the gift that is left in the room of each new guest. 

- it has a class attribute total_number_of_villas, which will keep track of the total number of villa objects that are currently "alive"
    - it's defined outside (before) method defs and lives in the class namespace (not instance namespace!)
    - to access it inside a method (which is in instance namespace) you need to prepend the class name (to get it from its class namespace)
    - in init, this counter gets incremented, one more instance was added
    - how to deal with villas being deleted? overwrite the __del__ method

In [34]:
from datetime import date # later needed for date representation (can do time math)

class villa(object): # should be Villa!
    ''' class to model a villa'''
    total_number_of_villas = 0 # class attribute, shared by all instances!
    def __init__(self, name):
        ''' name and reservation object of the villa'''
        self.villaName = name
        self.id = None # will be set later
        self.personalAssistant = None 
        villa.total_number_of_villas += 1
        print("Current total of", villa.total_number_of_villas, "villa instances")
    def setPersonalAssistant(self, pa):
        self.personalAssistant = pa
        print(f"Your personal assistant {pa} will be on call from 8.00am to 8.00pm for villa {self.villaName}") 
    def isVIP(self):
        # is self a vipVilla instance?
        return isinstance(self, vipVilla)
    def __del__(self):
        '''triggered on deletion'''
        villa.total_number_of_villas -= 1
        print("Current total of", villa.total_number_of_villas, "villa instances")
    def __str__(self):
        s =  f"Name: {self.villaName}, reservation id: {self.id}, PA: {self.personalAssistant}"
        if self.isVIP(): s += " (VIP)"
        return s
    

In [36]:
# test usage example:
v = villa("testvilla") # reservation id won't be used for now
v.setPersonalAssistant("Manuel") # <= from Faulty Towers!

# Note that v.isVIP()) and print(v) won't work yet, b/c we haven't defined the vipVilla class

Current total of 2 villa instances
Your personal assistant Manuel will be on call from 8.00am to 8.00pm for villa testvilla


### vipVilla
- inherits all attributes from class villa. (`villa.__init__`)
- also get all the villa methods but overwrites setPersonalAssistant()
- note that the attribute for teh PA is the same (personalAssistant), but we can alwasys check if this villa has as VIP PA, b/c of the isVIP() method the villa baseclass defined

In [9]:
class vipVilla(villa):
    def __init__(self, nn):
        villa.__init__(self, nn) # call baseclass init
    def setPersonalAssistant(self, pa):
        self.personalAssistant = pa # same attribute as for normal villa
        print(f"Your VIP assistant {pa} will be on call (7.00am-9.00pm) in " + #
              f"{self.villaName} to really pamper the crap out of you!")                

In [37]:
# test usage example - same as above but with vipVilla
vip = vipVilla("testVIPvilla") # reservation id won't be used for now
vip.setPersonalAssistant("Hervé Villechaize") # 

Current total of 3 villa instances
Your VIP assistant Hervé Villechaize will be on call (7.00am-9.00pm) in testVIPvilla to really pamper the crap out of you!


In [38]:
# get info on each villa
print(v)
print(vip)

Name: testvilla, reservation id: None, PA: Manuel
Name: testVIPvilla, reservation id: None, PA: Hervé Villechaize (VIP)


In [40]:
# isinstance() - what class does each instance belong to?
print(v.isVIP(), vip.isVIP()) # now even the baseclass can run isVIP()
isinstance(vip, villa) # also, vip (superclass) is an instance of it's baseclass

False True


False

### guest 

encapsulates these attribute
- first and last name
- number of adults
- number of children in the room. 
- access methods to last name (set/get)
- can print out its info

In [41]:
class guest(object):
    def __init__(self, first, last, numAdults, numChildren):
       self.first = first
       self.last = last
       self.noofAdults = numAdults
       self.noofChildren = numChildren
    def __str__(self): 
        return f'Guest name: {self.first}, {self.last}, {self.noofAdults} adults, {self.noofChildren} kids'

In [42]:
# test
g1 = guest("Ballard", "Berkeley", 1, 0)
print(g1)

g2 = guest("Fred", "Flintstone", 3, 1)
print(g2)

# keep a list of guests
guest_list = [g1, g2]

Guest name: Ballard, Berkeley, 1 adults, 0 kids
Guest name: Fred, Flintstone, 3 adults, 1 kids


### Reservations
- this is a non-physical class, meant to model a reservation system
- creating a reservation reserves a villa for a time (don't know the guest yet!
- uses a reservation id so humans have a handle on the reservation object
- attributes:
    - villa object (better than a name string, b/c that ensures we use an __existing__ villa!)
    - checkin date 
    - check out date, calculated from check in date + number of days to stay
    - reservation ID
- reservation id could be anything (string, int) and has to be set after __init__ (why?)
- can print out its data

In [43]:
from datetime import timedelta

class reservation(object):
    def __init__(self, villa_object, res_id, check_in_date, stay_num_days):
         self.checkinDate = check_in_date
         self.lengthofStay = stay_num_days
         self.villa = villa_object
         self.checkoutDate = check_in_date + timedelta(days=stay_num_days)
         self.reservID = res_id  
         self.villa.id = res_id # need to also set the red_id in the villa

    def getID(self):
            return self.self.reservID
    def getvillaName(self):
             return self.villaName
    def getcheckinDate(self):
            return self.checkinDate
    def getcheckoutDate(self):
            return self.checkoutDate
    def __str__(self):
       return f"{self.villa.villaName}: id {self.reservID}: reserved from {self.checkinDate}, for {self.lengthofStay} days, until {self.checkoutDate} "

In [44]:
# test for reservation
r1 = reservation(v, 123, date(2020,6,2), 4) # reserve slob villa for 4 days
print(r1)

r2 = reservation(vip, 456, date(2020,6,12), 21) # reserve VIP villa for 21 days
print(r2)

testvilla: id 123: reserved from 2020-06-02, for 4 days, until 2020-06-06 
testVIPvilla: id 456: reserved from 2020-06-12, for 21 days, until 2020-07-03 


### Resort class
- acts as a central "wrapper" around  all other classes
- an booking app would primarily interact with this object
- attributes:
    - name
    - list of its villas 
- can connect a guest to a reservation (which already links to a villa) via book() method
- can print out its current data (internal state)  
- I choose a dictionary to manifest a booking action:
    - I assign a reservation object as value (which reserves a specific villa for a duration) to an already existing guest object as key (objects are fine for keys, as long as the are immutable)

In [45]:
class resort(object):
    def __init__(self, resort_name, villa_list):
        self.resort_name = resort_name
        self.villa_list = villa_list
        self.booking_dict = {} # connects guest to their reservation and villa
                               # means that no guest can book 2 villas 
    def book(self, guest, reservation):
        self.booking_dict[guest] = reservation # reservation already links to villa!
    def get_reservation(self, guest):
        return self.booking_dict[guest]
    def __str__(self):
        s = self.resort_name + "\n"
        if len(self.villa_list) > 0:
            s += "villa list:\n"
            for v in self.villa_list:
                s += "\t" + str(v) + "\n"

        guestList = self.booking_dict.keys()
        if len(guestList) > 0:
            s += "guest list\n"
            for g in guestList:
                s += "\t" + str(g) + "\n"

            res_list = self.booking_dict.values()
            s += "reservation list\n"
            for r in res_list:
                s += "\t" + str(r) + "\n"

        return s

In [46]:
# make resort and add the 2 villas
rs = resort("myresort", [v, vip])
print(rs)

myresort
villa list:
	Name: testvilla, reservation id: 123, PA: Manuel
	Name: testVIPvilla, reservation id: 456, PA: Hervé Villechaize (VIP)



In [47]:
# book the 2 guests  into a villa using a reservation object
rs.book(g1, r1)
rs.book(g2, r2)
print(rs)

myresort
villa list:
	Name: testvilla, reservation id: 123, PA: Manuel
	Name: testVIPvilla, reservation id: 456, PA: Hervé Villechaize (VIP)
guest list
	Guest name: Ballard, Berkeley, 1 adults, 0 kids
	Guest name: Fred, Flintstone, 3 adults, 1 kids
reservation list
	testvilla: id 123: reserved from 2020-06-02, for 4 days, until 2020-06-06 
	testVIPvilla: id 456: reserved from 2020-06-12, for 21 days, until 2020-07-03 



In [48]:
# what villa has g2 reserved?  get the reservation for it and print it out
print(g2, rs.get_reservation(g2))

Guest name: Fred, Flintstone, 3 adults, 1 kids testVIPvilla: id 456: reserved from 2020-06-12, for 21 days, until 2020-07-03 


### Closing thoughts
- We could think of this as the core of a management app but there are many things missing (or could be improved)
- E.g. it should be possible to use the reservation id (confirmation code) to connect a guest to a reservation
- It's very likely that I will use this setup this (cleaned up some more) as you last python refresher exercise
